In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
basedir = './국토교통부_건축물대장_표제부/'
filenames = [f for f in os.listdir(basedir) if f.endswith('.txt')]
filenames

['mart_djy_03_2014.txt',
 'mart_djy_03_2016.txt',
 'mart_djy_03_2017.txt',
 'mart_djy_03_2018.txt',
 'mart_djy_03_2019.txt',
 'mart_djy_03_2020.txt',
 'MART_DJY_TITLE_2015.txt',
 'mart_shtreg_03.txt']

In [4]:
prac_df = pd.read_csv(basedir + filenames[-1], sep='|')
print(prac_df.shape)
prac_df.head(1)

c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (13,14,15,33,36,39,60,61,62,63) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(7389045, 2)


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,ORACLEDUMP\t,MGM_BLDRGST_PK\tREGSTR_GB_CD\tREGSTR_GB_CD_NM\tREGSTR_KIND_CD\tREGSTR_KIND_CD_NM\tPLAT_PLC\tNEW_PLAT_PLC\tBLD_NM\tSIGUNGU_CD\tBJDONG_CD\tPLAT_GB_CD\tBUN\tJI\tSPLOT_NM\tBLOCK\tLOT\tBYLOT_CNT\tETC_RCD_ITEM\tNA_ROAD_CD\tNA_BJDONG_CD\tNA_UGRND_CD\tNA_MAIN_BUN\tNA_SUB_BUN\tDONG_NM\tMAIN_ATCH_GB_CD\tMAIN_ATCH_GB_CD_NM\tSPCMT\tPLAT_AREA\tARCH_AREA\tBC_RAT\tTOTAREA\tVL_RAT_ESTM_TOTAREA\tVL_RAT\tSTRCT_CD\tSTRCT_CD_NM\tETC_STRCT\tMAIN_PURPS_CD\tMAIN_PURPS_CD_NM\tETC_PURPS\tROOF_CD\tROOF_CD_NM\tETC_ROOF\tHHLD_CNT\tFMLY_CNT\tHEIT\tGRND_FLR_CNT\tUGRND_FLR_CNT\tRIDE_USE_ELVT_CNT\tEMGEN_USE_ELVT_CNT\tATCH_BLD_CNT\tATCH_BLD_AREA\tTOT_DONG_TOTAREA\tINDR_MECH_UTCNT\tINDR_MECH_AREA\tOUDR_MECH_UTCNT\tOUDR_MECH_AREA\tINDR_AUTO_UTCNT\tINDR_AUTO_AREA\tOUDR_AUTO_UTCNT\tOUDR_AUTO_AREA\tPMS_DAY\tSTCNS_DAY\tUSEAPR_DAY\tPMSNO_YEAR\tPMSNO_KIK_CD\tPMSNO_KIK_CD_NM\tPMSNO_GB_CD\tPMSNO_GB_CD_NM\tHO_CNT\tENGR_GRADE\tENGR_RAT\tENGR_EPI\tGN_BLD_GRADE\tGN_BLD_CERT\tITG_BLD_GRADE\tITG_BLD_CERT\tCRTN_DAY
11305-10019310902,2,집합,3,표제부,서울특별시 강북구 수유동 484-80번지,서울특별시강북구 삼양로77가길 36,삼일교회,11305,10300.0,0.0,484.0,80.0,NaN,NaN,NaN,1,NaN,1.130541e+11,10301.0,0,36.0,0.0,삼일교회,0.0,주건축물,NaN,393.0,233.81,59.49,761.29,761.29,193.71,21.0,철근콘크리트구조,철근콘크리트구조(조적조(4층)),04000,제2종근린생활시설,"근린생활시설, 공동주택(다세대주택)",10.0,(철근)콘크리트,(철근)콘크리트(평스라브),3,0,12.2,4,0,0,0,0,0.0,761.29,0,0.0,0,0.0,0,0.0,7,80.5,20070821,20070829,20101216.0,2007,3080079.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101228


In [13]:
for f in filenames:
    year = f.replace('.txt', '')[-4:]
    
    try:
        prac_chunks = pd.read_csv(basedir + f, sep='|', chunksize=5)
        prac_df = next(prac_chunks)
    except:
        try:
            prac_chunks = pd.read_csv(basedir + f, sep='|', chunksize=5, encoding='euc-kr')
            prac_df = next(prac_chunks)
        except:
            try:
                prac_chunks = pd.read_csv(basedir + f, sep='|', chunksize=5, encoding='ansi')
                prac_df = next(prac_chunks)
            except:
                raise ValueError('myerror')
    
    if prac_df.shape[1] == 2:
        weird_string = prac_df.columns[0][-1]
        colnames = prac_df.columns[1].split(weird_string)
        print(len(colnames))
        
        try:
            prac_chunks = pd.read_csv(basedir + f, sep='|', chunksize=5, header=0, names=colnames)
            prac_df = next(prac_chunks)
        except:
            try:
                prac_chunks = pd.read_csv(basedir + f, sep='|', chunksize=5, encoding='euc-kr', header=0, names=colnames)
                prac_df = next(prac_chunks)
            except:
                try:
                    prac_chunks = pd.read_csv(basedir + f, sep='|', chunksize=5, encoding='ansi', header=0, names=colnames)
                    prac_df = next(prac_chunks)
                except:
                    raise ValueError('myerror')
        
        
    
    print(prac_df.shape)

77
(5, 77)
(5, 77)
77
(5, 77)
(5, 75)
(5, 75)
(5, 77)
(5, 77)


In [16]:
def create_bdinfo_csv():
    basedir = './국토교통부_건축물대장_표제부/'
    filenames = [f for f in os.listdir(basedir) if f.endswith('.txt')]
    
    for f in tqdm(filenames, position=0):
        year = f.replace('.txt', '')[-4:]
        print(year)

        try:
            prac_chunks = pd.read_csv(basedir + f, sep='|', chunksize=5)
            prac_df = next(prac_chunks)
        except:
            try:
                prac_chunks = pd.read_csv(basedir + f, sep='|', chunksize=5, encoding='euc-kr')
                prac_df = next(prac_chunks)
            except:
                try:
                    prac_chunks = pd.read_csv(basedir + f, sep='|', chunksize=5, encoding='ansi')
                    prac_df = next(prac_chunks)
                except:
                    raise ValueError('myerror')

        if prac_df.shape[1] == 2:
            weird_string = prac_df.columns[0][-1]
            colnames = prac_df.columns[1].split(weird_string)

            try:
                df = pd.read_csv(basedir + f, sep='|', header=0, names=colnames)
            except:
                try:
                    df = pd.read_csv(basedir + f, sep='|', encoding='euc-kr', header=0, names=colnames)
                except:
                    try:
                        df = pd.read_csv(basedir + f, sep='|', encoding='ansi', header=0, names=colnames)
                    except:
                        raise ValueError('myerror')
            
        else:
            try:
                df = pd.read_csv(basedir + f, sep='|', header=None)
            except:
                try:
                    df = pd.read_csv(basedir + f, sep='|', encoding='euc-kr', header=None)
                except:
                    try:
                        df = pd.read_csv(basedir + f, sep='|', encoding='ansi', header=None)
                    except:
                        raise ValueError('myerror')
                        
        print('file reading complete')                
        
        df['시'] = df.iloc[:,5].apply(lambda x: x.split(' ')[0] if pd.isna(x) == False else x)
        seoul_df = df[df['시'] == '서울특별시']
        
        seoul_df.to_csv('./국토교통부_건축물대장_표제부/bdinfo_{}.csv'.format(year), index=False)
        print('file writing complete')

In [17]:
create_bdinfo_csv()

  0%|                                                                                            | 0/7 [00:00<?, ?it/s]

2014


c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (13,14,15,33,36,39,60,61,62,63) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


file reading complete


 14%|████████████                                                                        | 1/7 [01:00<06:04, 60.72s/it]

...

file writing complete
2015


c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (14,15,33,36,39,60,61,62,63,64,69) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


file reading complete


100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [09:19<00:00, 79.98s/it]

file writing complete


In [18]:
filenames = [f for f in os.listdir(basedir) if f.endswith('.csv')]
filenames

['bdinfo_2014.csv',
 'bdinfo_2015.csv',
 'bdinfo_2016.csv',
 'bdinfo_2017.csv',
 'bdinfo_2018.csv',
 'bdinfo_2019.csv',
 'bdinfo_2020.csv']

In [21]:
colname_file = [f for f in os.listdir(basedir) if f.endswith('.xls')]
colname_file

['데이터구조.xls']

In [27]:
colname_df = pd.read_excel(basedir + colname_file[0], header=1)
print(colname_df.shape)
colname_df.head()

(77, 3)


,컬럼 한글명,데이터 타입,비고
0,관리_건축물대장_PK,VARCHAR(33),NaN
1,대장_구분_코드,VARCHAR(1),NaN
2,대장_구분_코드_명,VARCHAR(100),NaN
3,대장_종류_코드,VARCHAR(1),NaN
4,대장_종류_코드_명,VARCHAR(100),NaN


In [32]:
colnames = colname_df['컬럼 한글명'].tolist()
colnames

['관리_건축물대장_PK',
 '대장_구분_코드',
 '대장_구분_코드_명',
 '대장_종류_코드',
 '대장_종류_코드_명',
 '대지_위치',
 '도로명_대지_위치',
 '건물_명',
 '시군구_코드',
 '법정동_코드',
 '대지_구분_코드',
 '번',
 '지',
 '특수지_명',
 '블록',
 '로트',
 '외필지_수',
 '새주소_도로_코드',
 '새주소_법정동_코드',
 '새주소_지상지하_코드',
 '새주소_본_번',
 '새주소_부_번',
 '동_명',
 '주_부속_구분_코드',
 '주_부속_구분_코드_명',
 '대지_면적(㎡)',
 '건축_면적(㎡)',
 '건폐_율(%)',
 '연면적(㎡)',
 '용적_률_산정_연면적(㎡)',
 '용적_률(%)',
 '구조_코드',
 '구조_코드_명',
 '기타_구조',
 '주_용도_코드',
 '주_용도_코드_명',
 '기타_용도',
 '지붕_코드',
 '지붕_코드_명',
 '기타_지붕',
 '세대_수(세대)',
 '가구_수(가구)',
 '높이(m)',
 '지상_층_수',
 '지하_층_수',
 '승용_승강기_수',
 '비상용_승강기_수',
 '부속_건축물_수',
 '부속_건축물_면적(㎡)',
 '총_동_연면적(㎡)',
 '옥내_기계식_대수(대)',
 '옥내_기계식_면적(㎡)',
 '옥외_기계식_대수(대)',
 '옥외_기계식_면적(㎡)',
 '옥내_자주식_대수(대)',
 '옥내_자주식_면적(㎡)',
 '옥외_자주식_대수(대)',
 '옥외_자주식_면적(㎡)',
 '허가_일',
 '착공_일',
 '사용승인_일',
 '허가번호_년',
 '허가번호_기관_코드',
 '허가번호_기관_코드_명',
 '허가번호_구분_코드',
 '허가번호_구분_코드_명',
 '호_수(호)',
 '에너지효율_등급',
 '에너지절감_율',
 '에너지_EPI점수',
 '친환경_건축물_등급',
 '친환경_건축물_인증점수',
 '지능형_건축물_등급',
 '지능형_건축물_인증점수',
 '생성_일자',
 '내진_설계_적용_여부',
 '내진_능력']

In [31]:
for f in filenames:
    df = pd.read_csv(basedir + f)

(3, 77)


,MGM_BLDRGST_PK,REGSTR_GB_CD,REGSTR_GB_CD_NM,REGSTR_KIND_CD,REGSTR_KIND_CD_NM,PLAT_PLC,NEW_PLAT_PLC,BLD_NM,SIGUNGU_CD,BJDONG_CD,PLAT_GB_CD,BUN,JI,SPLOT_NM,BLOCK,LOT,BYLOT_CNT,ETC_RCD_ITEM,NA_ROAD_CD,NA_BJDONG_CD,NA_UGRND_CD,NA_MAIN_BUN,NA_SUB_BUN,DONG_NM,MAIN_ATCH_GB_CD,MAIN_ATCH_GB_CD_NM,SPCMT,PLAT_AREA,ARCH_AREA,BC_RAT,TOTAREA,VL_RAT_ESTM_TOTAREA,VL_RAT,STRCT_CD,STRCT_CD_NM,ETC_STRCT,MAIN_PURPS_CD,MAIN_PURPS_CD_NM,ETC_PURPS,ROOF_CD,ROOF_CD_NM,ETC_ROOF,HHLD_CNT,FMLY_CNT,HEIT,GRND_FLR_CNT,UGRND_FLR_CNT,RIDE_USE_ELVT_CNT,EMGEN_USE_ELVT_CNT,ATCH_BLD_CNT,ATCH_BLD_AREA,TOT_DONG_TOTAREA,INDR_MECH_UTCNT,INDR_MECH_AREA,OUDR_MECH_UTCNT,OUDR_MECH_AREA,INDR_AUTO_UTCNT,INDR_AUTO_AREA,OUDR_AUTO_UTCNT,OUDR_AUTO_AREA,PMS_DAY,STCNS_DAY,USEAPR_DAY,PMSNO_YEAR,PMSNO_KIK_CD,PMSNO_KIK_CD_NM,PMSNO_GB_CD,PMSNO_GB_CD_NM,HO_CNT,ENGR_GRADE,ENGR_RAT,ENGR_EPI,GN_BLD_GRADE,GN_BLD_CERT,ITG_BLD_GRADE,ITG_BLD_CERT,CRTN_DAY
0,11305-10019310902,2,집합,3,표제부,서울특별시 강북구 수유동 484-80번지,서울특별시강북구 삼양로77가길 36,삼일교회,11305,10300.0,0.0,484.0,80.0,NaN,NaN,NaN,1,NaN,1.130541e+11,10301.0,0,36.0,0.0,삼일교회,0.0,주건축물,NaN,393.00,233.81,59.49,761.29,761.29,193.71,21.0,철근콘크리트구조,철근콘크리트구조(조적조(4층)),4000,제2종근린생활시설,"근린생활시설, 공동주택(다세대주택)",10.0,(철근)콘크리트,(철근)콘크리트(평스라브),3,0,12.2,4,0,0,0,0,0.0,761.29,0,0.0,0,0.0,0,0.0,7,80.5,20070821.0,20070829.0,20101216.0,2007.0,3080079.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101228
1,11440-2163902,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,NaN,1.144041e+11,12001.0,0,61.0,10.0,NaN,0.0,주건축물,NaN,177.62,105.30,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,2000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.0,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.0,4,46.0,20030628.0,20030820.0,20040323.0,2003.0,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101230
2,11200-2360202,1,일반,2,일반건축물,서울특별시 성동구 송정동 73-566번지,서울특별시성동구 송정14길 17,대한예수교장로회비전교회,11200,11800.0,0.0,73.0,566.0,NaN,NaN,NaN,0,NaN,1.120041e+11,11801.0,0,17.0,0.0,NaN,0.0,주건축물,NaN,806.00,376.05,46.66,1185.07,797.98,99.00,21.0,철근콘크리트구조,"철근콘크리트,철골조",6000,종교시설,교회,10.0,(철근)콘크리트,"슬라브,경량철골조",0,0,13.2,2,1,0,0,0,0.0,1185.07,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,19781205.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110131


(3, 77)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76
0,11470-12121,2,집합,3,표제부,서울특별시 양천구 목동 318-41번지,서울특별시 양천구 목동중앙남로16길 48,아주하이빌,11470,10200.0,0.0,318.0,41.0,NaN,NaN,NaN,0,NaN,1.147041e+11,10201.0,0,48.0,0.0,NaN,0.0,주건축물,NaN,261.0,155.82,59.7,605.000,605.0,231.8,21.0,철근콘크리트구조,철근콘크리트조,2000,공동주택,다세대주택,10.0,(철근)콘크리트,평스라브,8,0,14.45,5,0,0,0,0,0.0,605.000,0,0.0,0,0.0,4,51.93,2,23.0,20020409,20020513.0,20020912,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110415
1,11710-2809,2,집합,3,표제부,서울특별시 송파구 방이동 89번지,서울특별시 송파구 양재대로 1218,올림픽선수기자촌아파트,11710,11100.0,0.0,89.0,0.0,NaN,NaN,NaN,2,NaN,1.171020e+11,11102.0,0,1218.0,0.0,314동,0.0,주건축물,NaN,0.0,0.00,0.0,4632.720,0.0,0.0,21.0,철근콘크리트구조,철근콘크리트조,2000,공동주택,아파트,10.0,(철근)콘크리트,스라브,28,0,0.00,14,1,0,0,0,0.0,4632.720,0,0.0,0,0.0,0,0.00,0,0.0,19870214,NaN,19880614,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110415
2,11710-2885,2,집합,3,표제부,서울특별시 송파구 방이동 89번지,서울특별시 송파구 양재대로 1218,올림픽선수기자촌아파트,11710,11100.0,0.0,89.0,0.0,NaN,NaN,NaN,2,NaN,1.171020e+11,11102.0,0,1218.0,0.0,106동,0.0,주건축물,NaN,0.0,0.00,0.0,6980.884,0.0,0.0,21.0,철근콘크리트구조,철근콘크리트조,2000,공동주택,아파트,10.0,(철근)콘크리트,스라브,48,0,0.00,12,1,0,0,0,0.0,6980.884,0,0.0,0,0.0,0,0.00,0,0.0,19870214,NaN,19880614,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110415


(3, 77)


...

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74
0,11290-12818,1,일반,2,일반건축물,서울특별시 성북구 안암동3가 105번지,서울특별시 성북구 인촌로8길 75,NaN,11290,12300.0,0.0,105,0,NaN,NaN,NaN,0,1.129041e+11,12302.0,0,75.0,0.0,NaN,0.0,주건축물,0.0,0.00,0.00,26.45,23.14,0.00,51.0,일반목구조,"일반목구조,기타콘크리트구조",1000,단독주택,주택,20.0,기와,기와,0,1,0.0,1,1,0,0,1,6.62,26.45,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20120222
1,11545-12332,1,일반,2,일반건축물,서울특별시 금천구 시흥동 791-31번지,서울특별시 금천구 독산로44다길 9-1,NaN,11545,10300.0,0.0,791,31,NaN,NaN,NaN,0,1.154542e+11,10301.0,0,9.0,1.0,NaN,0.0,주건축물,99.2,58.46,58.93,149.58,102.12,102.94,11.0,벽돌구조,연와조,1000,단독주택,다가구용 단독주택(3가구),10.0,(철근)콘크리트,평스라브,0,3,7.6,2,1,0,0,0,0.00,149.58,0,0.0,0,0.0,0,0.0,0,0.0,19930222.0,19930430.0,19930818.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20120724
2,11530-23068,1,일반,2,일반건축물,서울특별시 구로구 신도림동 296-69번지,서울특별시 구로구 신도림로15가길 2,NaN,11530,10100.0,0.0,296,69,NaN,NaN,NaN,0,1.153041e+11,10101.0,0,2.0,0.0,NaN,0.0,주건축물,0.0,34.71,0.00,34.71,34.71,0.00,12.0,블록구조,세멘부럭조,1000,단독주택,주택,20.0,기와,세멘와즙,0,1,0.0,1,0,0,0,0,0.00,34.71,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,19651001.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20120530


(3, 75)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74
0,11260-21410,1,일반,2,일반건축물,서울특별시 중랑구 중화동 441-2번지,서울특별시 중랑구 봉화산로33길 35,NaN,11260,10300.0,0.0,441,2,NaN,NaN,NaN,0,1.126041e+11,10301.0,0.0,35.0,0.0,NaN,0.0,주건축물,209.0,124.94,59.78,476.31,353.46,169.12,21.0,철근콘크리트구조,철근콘크리트구조,1000,단독주택,단독주택,10.0,(철근)콘크리트,(철근)콘크리트 평슬라브,0,4,10.90,3,1,0,0,0,0.0,0.00,0,0.0,0,0.0,4,122.85,0,0.0,20030117,20030217,20030707.0,2003.0,3060079.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20120728
1,11410-17271,2,집합,3,표제부,서울특별시 서대문구 연희동 151-82번지,서울특별시 서대문구 연희로31길 19-3,NaN,11410,11700.0,0.0,151,82,NaN,NaN,NaN,0,1.141041e+11,11701.0,0.0,19.0,3.0,NaN,0.0,주건축물,188.0,111.93,59.54,532.24,398.45,211.94,21.0,철근콘크리트구조,철근콘크리트조,2000,공동주택,다세대주택,10.0,(철근)콘크리트,평스라브,4,0,11.75,4,1,0,0,0,0.0,532.24,0,0.0,0,0.0,4,133.79,0,0.0,20000419,20000421,20001027.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20170222
2,11500-18632,1,일반,2,일반건축물,서울특별시 강서구 화곡동 166-33번지,서울특별시 강서구 강서로18길 125,화곡동상가주택,11500,10300.0,0.0,166,33,NaN,NaN,NaN,0,1.150041e+11,10301.0,0.0,125.0,0.0,화곡동상가주택,0.0,주건축물,281.9,159.59,56.61,482.05,433.28,153.70,21.0,철근콘크리트구조,철근콘크리트조및조적조,1000,단독주택,근린생활시설및다가구주택(4가구),10.0,(철근)콘크리트,평스라브,0,4,10.50,3,1,0,0,0,0.0,147.28,0,0.0,0,0.0,0,0.00,4,46.0,19921022,19921027,19930413.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20170222


(3, 77)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76
0,11500-25658,1,일반,2,일반건축물,서울특별시 강서구 화곡동 366-90번지,서울특별시 강서구 가로공원로80가길 34,NaN,11500,10300.0,0.0,366,90,NaN,NaN,NaN,0,1.150041e+11,10301.0,0.0,34.0,0.0,NaN,0.0,주건축물,141.40,69.75,49.32,209.25,139.50,98.67,11.0,벽돌구조,연와조,1000,단독주택,단독주택,10.0,(철근)콘크리트,평스라브 일부 기와,0,1,7.90,2,1,0,0,0,0.0,209.25,0,0.0,0,0.0,0,0.00,0,0.0,19900321,19900521.0,19901116.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20181204,0.0,NaN
1,11200-21285,1,일반,2,일반건축물,서울특별시 성동구 성수동2가 301-27번지,,NaN,11200,11500.0,0.0,301,27,NaN,NaN,NaN,0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,주건축물,90.42,0.00,0.00,206.12,0.00,0.00,21.0,철근콘크리트구조,철근콘크리트 연와조,1000,단독주택,다가구주택(4가구),10.0,(철근)콘크리트,평슬라브위주위기와,0,4,0.00,3,1,0,0,0,0.0,206.12,0,0.0,0,0.0,0,0.00,0,0.0,19900718,NaN,19901220.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20181205,0.0,NaN
2,11170-22196,2,집합,3,표제부,서울특별시 용산구 한강로2가 83-4번지,서울특별시 용산구 한강대로46길 25,NaN,11170,12500.0,0.0,83,4,NaN,NaN,NaN,0,1.117041e+11,12501.0,0.0,25.0,NaN,NaN,0.0,주건축물,219.90,119.72,54.44,536.88,436.51,198.50,21.0,철근콘크리트구조,철근콘크리트구조,4000,제2종근린생활시설,제1.2종근린생활시설,10.0,(철근)콘크리트,평스라브,13,0,13.35,5,1,0,0,0,0.0,536.88,0,0.0,0,0.0,2,39.91,2,23.0,20070215,20070226.0,20070731.0,2007.0,3020078.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20191004,1.0,NaN


In [43]:
correct_cols_77 = [
    colnames[0],
    colnames[1],
    colnames[2],
    colnames[3],
    colnames[4],
    colnames[5],
    colnames[6],
    colnames[7],
    colnames[8],
    colnames[9],
    colnames[10],
    colnames[11],
    colnames[12],
    colnames[13],
    colnames[14],
    colnames[15],
    colnames[16],
    'ETC_RCD_ITEM_(unknown)',
    colnames[17],
    colnames[18],
    colnames[19],
    colnames[20],
    colnames[21],
    colnames[22],
    colnames[23],
    colnames[24],
    'SPCMT_(unknown)',
    colnames[25],
    colnames[26],
    colnames[27],
    colnames[28],
    colnames[29],
    colnames[30],
    colnames[31],
    colnames[32],
    colnames[33],
    colnames[34],
    colnames[35],
    colnames[36],
    colnames[37],
    colnames[38],
    colnames[39],
    colnames[40],
    colnames[41],
    colnames[42],
    colnames[43],
    colnames[44],
    colnames[45],
    colnames[46],
    colnames[47],
    colnames[48],
    colnames[49],
    colnames[50],
    colnames[51],
    colnames[52],
    colnames[53],
    colnames[54],
    colnames[55],
    colnames[56],
    colnames[57],
    colnames[58],
    colnames[59],
    colnames[60],
    colnames[61],
    colnames[62],
    colnames[63],
    colnames[64],
    colnames[65],
    colnames[66],
    colnames[67],
    colnames[68],
    colnames[69],
    colnames[70],
    colnames[71],
    colnames[72],
    colnames[73],
    colnames[74]
]

In [44]:
correct_cols_75 = [
    colnames[0],
    colnames[1],
    colnames[2],
    colnames[3],
    colnames[4],
    colnames[5],
    colnames[6],
    colnames[7],
    colnames[8],
    colnames[9],
    colnames[10],
    colnames[11],
    colnames[12],
    colnames[13],
    colnames[14],
    colnames[15],
    colnames[16],
    colnames[17],
    colnames[18],
    colnames[19],
    colnames[20],
    colnames[21],
    colnames[22],
    colnames[23],
    colnames[24],
    colnames[25],
    colnames[26],
    colnames[27],
    colnames[28],
    colnames[29],
    colnames[30],
    colnames[31],
    colnames[32],
    colnames[33],
    colnames[34],
    colnames[35],
    colnames[36],
    colnames[37],
    colnames[38],
    colnames[39],
    colnames[40],
    colnames[41],
    colnames[42],
    colnames[43],
    colnames[44],
    colnames[45],
    colnames[46],
    colnames[47],
    colnames[48],
    colnames[49],
    colnames[50],
    colnames[51],
    colnames[52],
    colnames[53],
    colnames[54],
    colnames[55],
    colnames[56],
    colnames[57],
    colnames[58],
    colnames[59],
    colnames[60],
    colnames[61],
    colnames[62],
    colnames[63],
    colnames[64],
    colnames[65],
    colnames[66],
    colnames[67],
    colnames[68],
    colnames[69],
    colnames[70],
    colnames[71],
    colnames[72],
    colnames[73],
    colnames[74]
]

In [54]:
for f in filenames:
    chunks = pd.read_csv(basedir + f, chunksize=3)
    chunk = next(chunks)
    
    if chunk.shape[1] == 76:
        chunk = chunk.drop(columns=['시'])
        chunk.columns = correct_cols_75
        display(chunk)

,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자
0,11290-12818,1,일반,2,일반건축물,서울특별시 성북구 안암동3가 105번지,서울특별시 성북구 인촌로8길 75,NaN,11290,12300.0,0.0,105,0,NaN,NaN,NaN,0,1.129041e+11,12302.0,0,75.0,0.0,NaN,0.0,주건축물,0.0,0.00,0.00,26.45,23.14,0.00,51.0,일반목구조,"일반목구조,기타콘크리트구조",1000,단독주택,주택,20.0,기와,기와,0,1,0.0,1,1,0,0,1,6.62,26.45,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20120222
1,11545-12332,1,일반,2,일반건축물,서울특별시 금천구 시흥동 791-31번지,서울특별시 금천구 독산로44다길 9-1,NaN,11545,10300.0,0.0,791,31,NaN,NaN,NaN,0,1.154542e+11,10301.0,0,9.0,1.0,NaN,0.0,주건축물,99.2,58.46,58.93,149.58,102.12,102.94,11.0,벽돌구조,연와조,1000,단독주택,다가구용 단독주택(3가구),10.0,(철근)콘크리트,평스라브,0,3,7.6,2,1,0,0,0,0.00,149.58,0,0.0,0,0.0,0,0.0,0,0.0,19930222.0,19930430.0,19930818.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20120724
2,11530-23068,1,일반,2,일반건축물,서울특별시 구로구 신도림동 296-69번지,서울특별시 구로구 신도림로15가길 2,NaN,11530,10100.0,0.0,296,69,NaN,NaN,NaN,0,1.153041e+11,10101.0,0,2.0,0.0,NaN,0.0,주건축물,0.0,34.71,0.00,34.71,34.71,0.00,12.0,블록구조,세멘부럭조,1000,단독주택,주택,20.0,기와,세멘와즙,0,1,0.0,1,0,0,0,0,0.00,34.71,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,19651001.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20120530


,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자
0,11260-21410,1,일반,2,일반건축물,서울특별시 중랑구 중화동 441-2번지,서울특별시 중랑구 봉화산로33길 35,NaN,11260,10300.0,0.0,441,2,NaN,NaN,NaN,0,1.126041e+11,10301.0,0.0,35.0,0.0,NaN,0.0,주건축물,209.0,124.94,59.78,476.31,353.46,169.12,21.0,철근콘크리트구조,철근콘크리트구조,1000,단독주택,단독주택,10.0,(철근)콘크리트,(철근)콘크리트 평슬라브,0,4,10.90,3,1,0,0,0,0.0,0.00,0,0.0,0,0.0,4,122.85,0,0.0,20030117,20030217,20030707.0,2003.0,3060079.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20120728
1,11410-17271,2,집합,3,표제부,서울특별시 서대문구 연희동 151-82번지,서울특별시 서대문구 연희로31길 19-3,NaN,11410,11700.0,0.0,151,82,NaN,NaN,NaN,0,1.141041e+11,11701.0,0.0,19.0,3.0,NaN,0.0,주건축물,188.0,111.93,59.54,532.24,398.45,211.94,21.0,철근콘크리트구조,철근콘크리트조,2000,공동주택,다세대주택,10.0,(철근)콘크리트,평스라브,4,0,11.75,4,1,0,0,0,0.0,532.24,0,0.0,0,0.0,4,133.79,0,0.0,20000419,20000421,20001027.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20170222
2,11500-18632,1,일반,2,일반건축물,서울특별시 강서구 화곡동 166-33번지,서울특별시 강서구 강서로18길 125,화곡동상가주택,11500,10300.0,0.0,166,33,NaN,NaN,NaN,0,1.150041e+11,10301.0,0.0,125.0,0.0,화곡동상가주택,0.0,주건축물,281.9,159.59,56.61,482.05,433.28,153.70,21.0,철근콘크리트구조,철근콘크리트조및조적조,1000,단독주택,근린생활시설및다가구주택(4가구),10.0,(철근)콘크리트,평스라브,0,4,10.50,3,1,0,0,0,0.0,147.28,0,0.0,0,0.0,0,0.00,4,46.0,19921022,19921027,19930413.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20170222


In [51]:
prac_chunks1 = pd.read_csv(basedir + 'bdinfo_2018.csv', chunksize=3)
prac_df1 = next(prac_chunks1)
print(prac_df1.shape)
prac_df1.head()

(3, 76)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,시
0,11290-12818,1,일반,2,일반건축물,서울특별시 성북구 안암동3가 105번지,서울특별시 성북구 인촌로8길 75,NaN,11290,12300.0,0.0,105,0,NaN,NaN,NaN,0,1.129041e+11,12302.0,0,75.0,0.0,NaN,0.0,주건축물,0.0,0.00,0.00,26.45,23.14,0.00,51.0,일반목구조,"일반목구조,기타콘크리트구조",1000,단독주택,주택,20.0,기와,기와,0,1,0.0,1,1,0,0,1,6.62,26.45,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20120222,서울특별시
1,11545-12332,1,일반,2,일반건축물,서울특별시 금천구 시흥동 791-31번지,서울특별시 금천구 독산로44다길 9-1,NaN,11545,10300.0,0.0,791,31,NaN,NaN,NaN,0,1.154542e+11,10301.0,0,9.0,1.0,NaN,0.0,주건축물,99.2,58.46,58.93,149.58,102.12,102.94,11.0,벽돌구조,연와조,1000,단독주택,다가구용 단독주택(3가구),10.0,(철근)콘크리트,평스라브,0,3,7.6,2,1,0,0,0,0.00,149.58,0,0.0,0,0.0,0,0.0,0,0.0,19930222.0,19930430.0,19930818.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20120724,서울특별시
2,11530-23068,1,일반,2,일반건축물,서울특별시 구로구 신도림동 296-69번지,서울특별시 구로구 신도림로15가길 2,NaN,11530,10100.0,0.0,296,69,NaN,NaN,NaN,0,1.153041e+11,10101.0,0,2.0,0.0,NaN,0.0,주건축물,0.0,34.71,0.00,34.71,34.71,0.00,12.0,블록구조,세멘부럭조,1000,단독주택,주택,20.0,기와,세멘와즙,0,1,0.0,1,0,0,0,0,0.00,34.71,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,19651001.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20120530,서울특별시


In [55]:
for f in tqdm(filenames, position=0):
    df = pd.read_csv(basedir + f).drop(columns=['시'])
    
    if df.shape[1] == 75:
        df.columns = correct_cols_75
    elif df.shape[1] == 77:
        df.columns = correct_cols_77
        
    df.to_csv(basedir + f)

 14%|████████████                                                                        | 1/7 [00:25<02:31, 25.22s/it]c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (13,14,15,36,60,61,69) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 29%|████████████████████████                                                            | 2/7 [00:51<02:08, 25.78s/it]c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (14,15,60,61,69) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 43%|████████████████████████████████████                                                | 3/7 [01:16<01:41, 25.48s/it]c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (15,60,61,69) have mixed typ

In [56]:
for f in filenames:
    chunks = pd.read_csv(basedir + f, chunksize=3)
    df = next(chunks)
    display(df)

,Unnamed: 0,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,ETC_RCD_ITEM_(unknown),새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,SPCMT_(unknown),대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자
0,0,11305-10019310902,2,집합,3,표제부,서울특별시 강북구 수유동 484-80번지,서울특별시강북구 삼양로77가길 36,삼일교회,11305,10300.0,0.0,484.0,80.0,NaN,NaN,NaN,1,NaN,1.130541e+11,10301.0,0,36.0,0.0,삼일교회,0.0,주건축물,NaN,393.00,233.81,59.49,761.29,761.29,193.71,21.0,철근콘크리트구조,철근콘크리트구조(조적조(4층)),4000,제2종근린생활시설,"근린생활시설, 공동주택(다세대주택)",10.0,(철근)콘크리트,(철근)콘크리트(평스라브),3,0,12.2,4,0,0,0,0,0.0,761.29,0,0.0,0,0.0,0,0.0,7,80.5,20070821.0,20070829.0,20101216.0,2007.0,3080079.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101228
1,1,11440-2163902,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,NaN,1.144041e+11,12001.0,0,61.0,10.0,NaN,0.0,주건축물,NaN,177.62,105.30,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,2000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.0,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.0,4,46.0,20030628.0,20030820.0,20040323.0,2003.0,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101230
2,2,11200-2360202,1,일반,2,일반건축물,서울특별시 성동구 송정동 73-566번지,서울특별시성동구 송정14길 17,대한예수교장로회비전교회,11200,11800.0,0.0,73.0,566.0,NaN,NaN,NaN,0,NaN,1.120041e+11,11801.0,0,17.0,0.0,NaN,0.0,주건축물,NaN,806.00,376.05,46.66,1185.07,797.98,99.00,21.0,철근콘크리트구조,"철근콘크리트,철골조",6000,종교시설,교회,10.0,(철근)콘크리트,"슬라브,경량철골조",0,0,13.2,2,1,0,0,0,0.0,1185.07,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,19781205.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110131


,Unnamed: 0,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,ETC_RCD_ITEM_(unknown),새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,SPCMT_(unknown),대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자
0,0,11470-12121,2,집합,3,표제부,서울특별시 양천구 목동 318-41번지,서울특별시 양천구 목동중앙남로16길 48,아주하이빌,11470,10200.0,0.0,318.0,41.0,NaN,NaN,NaN,0,NaN,1.147041e+11,10201.0,0,48.0,0.0,NaN,0.0,주건축물,NaN,261.0,155.82,59.7,605.000,605.0,231.8,21.0,철근콘크리트구조,철근콘크리트조,2000,공동주택,다세대주택,10.0,(철근)콘크리트,평스라브,8,0,14.45,5,0,0,0,0,0.0,605.000,0,0.0,0,0.0,4,51.93,2,23.0,20020409,20020513.0,20020912.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110415
1,1,11710-2809,2,집합,3,표제부,서울특별시 송파구 방이동 89번지,서울특별시 송파구 양재대로 1218,올림픽선수기자촌아파트,11710,11100.0,0.0,89.0,0.0,NaN,NaN,NaN,2,NaN,1.171020e+11,11102.0,0,1218.0,0.0,314동,0.0,주건축물,NaN,0.0,0.00,0.0,4632.720,0.0,0.0,21.0,철근콘크리트구조,철근콘크리트조,2000,공동주택,아파트,10.0,(철근)콘크리트,스라브,28,0,0.00,14,1,0,0,0,0.0,4632.720,0,0.0,0,0.0,0,0.00,0,0.0,19870214,NaN,19880614.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110415
2,2,11710-2885,2,집합,3,표제부,서울특별시 송파구 방이동 89번지,서울특별시 송파구 양재대로 1218,올림픽선수기자촌아파트,11710,11100.0,0.0,89.0,0.0,NaN,NaN,NaN,2,NaN,1.171020e+11,11102.0,0,1218.0,0.0,106동,0.0,주건축물,NaN,0.0,0.00,0.0,6980.884,0.0,0.0,21.0,철근콘크리트구조,철근콘크리트조,2000,공동주택,아파트,10.0,(철근)콘크리트,스라브,48,0,0.00,12,1,0,0,0,0.0,6980.884,0,0.0,0,0.0,0,0.00,0,0.0,19870214,NaN,19880614.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110415


,Unnamed: 0,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,ETC_RCD_ITEM_(unknown),새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,SPCMT_(unknown),대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자
0,0,11470-10475,2,집합,3,표제부,서울특별시 양천구 목동 318-196번지,서울특별시 양천구 목동중앙본로7가길 41,목동다세대주택,11470,10200.0,0.0,318.0,196.0,NaN,NaN,NaN,0,NaN,1.147041e+11,10201.0,0,41.0,0.0,NaN,0.0,주건축물,NaN,228.0,136.73,59.97,591.66,460.37,201.91,21.0,철근콘크리트구조,철근콘크리트조,2000,공동주택,공동주택,10.0,(철근)콘크리트,철근콘크리트평스라브,9,0,12.4,4,1,0,0,0,0.0,591.66,0,0.0,0,0.0,4,60.5,2,23.0,20010615.0,20010719.0,20011130.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110415
1,1,11710-19384,1,일반,2,일반건축물,서울특별시 송파구 잠실동 193-9번지,서울특별시 송파구 올림픽로8길 21,NaN,11710,10100.0,0.0,193.0,9.0,NaN,NaN,NaN,0,NaN,1.171042e+11,10101.0,0,21.0,0.0,NaN,0.0,주건축물,NaN,620.0,294.93,47.57,1450.63,1155.43,186.36,21.0,철근콘크리트구조,철근콘크리트조,3000,제1종근린생활시설,근린생활시설,10.0,(철근)콘크리트,평스라브,0,0,0.0,4,1,0,0,0,0.0,1450.63,0,0.0,0,0.0,0,0.0,10,116.0,198608.0,NaN,19861216.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110415
2,2,11710-2809,2,집합,3,표제부,서울특별시 송파구 방이동 89번지,서울특별시 송파구 양재대로 1218,올림픽선수기자촌아파트,11710,11100.0,0.0,89.0,0.0,NaN,NaN,NaN,2,NaN,1.171020e+11,11102.0,0,1218.0,0.0,314동,0.0,주건축물,NaN,0.0,0.00,0.00,4632.72,0.00,0.00,21.0,철근콘크리트구조,철근콘크리트조,2000,공동주택,아파트,10.0,(철근)콘크리트,스라브,28,0,0.0,14,1,0,0,0,0.0,4632.72,0,0.0,0,0.0,0,0.0,0,0.0,19870214.0,NaN,19880614.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110415


,Unnamed: 0,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,ETC_RCD_ITEM_(unknown),새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,SPCMT_(unknown),대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자
0,0,11260-6591,1,일반,2,일반건축물,서울특별시 중랑구 면목동 184-34번지,서울특별시 중랑구 중랑천로10길 90,NaN,11260,10100.0,0.0,184,34,NaN,NaN,NaN,0,NaN,1.126041e+11,10101.0,0,90.0,0.0,NaN,0.0,주건축물,NaN,173.6,104.01,59.91,494.40,395.52,227.83,21.0,철근콘크리트구조,철근콘크리트조및연와조,1000,단독주택,근린생활및주택(1가구),10.0,(철근)콘크리트,평스라브,0,1,12.3,4,1,0,0,0,0.00,494.40,0,0.0,0,0.0,2,32.96,2,23.0,19940311.0,19940413.0,19940826.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20170413
1,1,11410-16240,1,일반,2,일반건축물,서울특별시 서대문구 연희동 444-115번지,서울특별시 서대문구 연희로11아길 20-24,NaN,11410,11700.0,0.0,444,115,NaN,NaN,NaN,0,NaN,1.141041e+11,11701.0,0,20.0,24.0,NaN,0.0,주건축물,NaN,0.0,0.00,0.00,153.42,0.00,0.00,11.0,벽돌구조,연와조,1000,단독주택,주택,10.0,(철근)콘크리트,슬라브,0,1,0.0,1,1,0,0,1,17.98,153.42,0,0.0,0,0.0,0,0.00,0,0.0,NaN,NaN,19750703.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20170413
2,2,11320-3775,1,일반,2,일반건축물,서울특별시 도봉구 쌍문동 532-1번지,서울특별시 도봉구 삼양로162가길 42-10,NaN,11320,10500.0,0.0,532,1,NaN,NaN,NaN,1,NaN,1.132041e+11,10501.0,0,42.0,10.0,NaN,0.0,주건축물,NaN,3242.6,828.00,25.54,828.00,828.00,25.54,32.0,경량철골구조,조립식(샌드위치단열판넬),18000,창고시설,창고 및 단독주택,90.0,기타지붕,기타지붕,0,1,4.4,1,0,0,0,0,0.00,828.00,0,0.0,0,0.0,0,0.00,6,68.0,19950424.0,19950508.0,19950819.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20170413


,Unnamed: 0,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자
0,0,11290-12818,1,일반,2,일반건축물,서울특별시 성북구 안암동3가 105번지,서울특별시 성북구 인촌로8길 75,NaN,11290,12300.0,0.0,105,0,NaN,NaN,NaN,0,1.129041e+11,12302.0,0,75.0,0.0,NaN,0.0,주건축물,0.0,0.00,0.00,26.45,23.14,0.00,51.0,일반목구조,"일반목구조,기타콘크리트구조",1000,단독주택,주택,20.0,기와,기와,0,1,0.0,1,1,0,0,1,6.62,26.45,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20120222
1,1,11545-12332,1,일반,2,일반건축물,서울특별시 금천구 시흥동 791-31번지,서울특별시 금천구 독산로44다길 9-1,NaN,11545,10300.0,0.0,791,31,NaN,NaN,NaN,0,1.154542e+11,10301.0,0,9.0,1.0,NaN,0.0,주건축물,99.2,58.46,58.93,149.58,102.12,102.94,11.0,벽돌구조,연와조,1000,단독주택,다가구용 단독주택(3가구),10.0,(철근)콘크리트,평스라브,0,3,7.6,2,1,0,0,0,0.00,149.58,0,0.0,0,0.0,0,0.0,0,0.0,19930222.0,19930430.0,19930818.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20120724
2,2,11530-23068,1,일반,2,일반건축물,서울특별시 구로구 신도림동 296-69번지,서울특별시 구로구 신도림로15가길 2,NaN,11530,10100.0,0.0,296,69,NaN,NaN,NaN,0,1.153041e+11,10101.0,0,2.0,0.0,NaN,0.0,주건축물,0.0,34.71,0.00,34.71,34.71,0.00,12.0,블록구조,세멘부럭조,1000,단독주택,주택,20.0,기와,세멘와즙,0,1,0.0,1,0,0,0,0,0.00,34.71,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,19651001.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20120530


,Unnamed: 0,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자
0,0,11260-21410,1,일반,2,일반건축물,서울특별시 중랑구 중화동 441-2번지,서울특별시 중랑구 봉화산로33길 35,NaN,11260,10300.0,0.0,441,2,NaN,NaN,NaN,0,1.126041e+11,10301.0,0.0,35.0,0.0,NaN,0.0,주건축물,209.0,124.94,59.78,476.31,353.46,169.12,21.0,철근콘크리트구조,철근콘크리트구조,1000,단독주택,단독주택,10.0,(철근)콘크리트,(철근)콘크리트 평슬라브,0,4,10.90,3,1,0,0,0,0.0,0.00,0,0.0,0,0.0,4,122.85,0,0.0,20030117,20030217.0,20030707.0,2003.0,3060079.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20120728
1,1,11410-17271,2,집합,3,표제부,서울특별시 서대문구 연희동 151-82번지,서울특별시 서대문구 연희로31길 19-3,NaN,11410,11700.0,0.0,151,82,NaN,NaN,NaN,0,1.141041e+11,11701.0,0.0,19.0,3.0,NaN,0.0,주건축물,188.0,111.93,59.54,532.24,398.45,211.94,21.0,철근콘크리트구조,철근콘크리트조,2000,공동주택,다세대주택,10.0,(철근)콘크리트,평스라브,4,0,11.75,4,1,0,0,0,0.0,532.24,0,0.0,0,0.0,4,133.79,0,0.0,20000419,20000421.0,20001027.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20170222
2,2,11500-18632,1,일반,2,일반건축물,서울특별시 강서구 화곡동 166-33번지,서울특별시 강서구 강서로18길 125,화곡동상가주택,11500,10300.0,0.0,166,33,NaN,NaN,NaN,0,1.150041e+11,10301.0,0.0,125.0,0.0,화곡동상가주택,0.0,주건축물,281.9,159.59,56.61,482.05,433.28,153.70,21.0,철근콘크리트구조,철근콘크리트조및조적조,1000,단독주택,근린생활시설및다가구주택(4가구),10.0,(철근)콘크리트,평스라브,0,4,10.50,3,1,0,0,0,0.0,147.28,0,0.0,0,0.0,0,0.00,4,46.0,19921022,19921027.0,19930413.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20170222


,Unnamed: 0,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,ETC_RCD_ITEM_(unknown),새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,SPCMT_(unknown),대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자
0,0,11500-25658,1,일반,2,일반건축물,서울특별시 강서구 화곡동 366-90번지,서울특별시 강서구 가로공원로80가길 34,NaN,11500,10300.0,0.0,366,90,NaN,NaN,NaN,0,1.150041e+11,10301.0,0.0,34.0,0.0,NaN,0.0,주건축물,141.40,69.75,49.32,209.25,139.50,98.67,11.0,벽돌구조,연와조,1000,단독주택,단독주택,10.0,(철근)콘크리트,평스라브 일부 기와,0,1,7.90,2,1,0,0,0,0.0,209.25,0,0.0,0,0.0,0,0.00,0,0.0,19900321,19900521.0,19901116.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20181204,0.0,NaN
1,1,11200-21285,1,일반,2,일반건축물,서울특별시 성동구 성수동2가 301-27번지,,NaN,11200,11500.0,0.0,301,27,NaN,NaN,NaN,0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,주건축물,90.42,0.00,0.00,206.12,0.00,0.00,21.0,철근콘크리트구조,철근콘크리트 연와조,1000,단독주택,다가구주택(4가구),10.0,(철근)콘크리트,평슬라브위주위기와,0,4,0.00,3,1,0,0,0,0.0,206.12,0,0.0,0,0.0,0,0.00,0,0.0,19900718,NaN,19901220.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20181205,0.0,NaN
2,2,11170-22196,2,집합,3,표제부,서울특별시 용산구 한강로2가 83-4번지,서울특별시 용산구 한강대로46길 25,NaN,11170,12500.0,0.0,83,4,NaN,NaN,NaN,0,1.117041e+11,12501.0,0.0,25.0,NaN,NaN,0.0,주건축물,219.90,119.72,54.44,536.88,436.51,198.50,21.0,철근콘크리트구조,철근콘크리트구조,4000,제2종근린생활시설,제1.2종근린생활시설,10.0,(철근)콘크리트,평스라브,13,0,13.35,5,1,0,0,0,0.0,536.88,0,0.0,0,0.0,2,39.91,2,23.0,20070215,20070226.0,20070731.0,2007.0,3020078.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20191004,1.0,NaN


In [57]:
for f in tqdm(filenames, position=0):
    df = pd.read_csv(basedir + f).drop(columns=['Unnamed: 0'])
        
    df.to_csv(basedir + f, index=False)

  0%|                                                                                            | 0/7 [00:00<?, ?it/s]c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (15,16,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 14%|████████████                                                                        | 1/7 [00:25<02:35, 25.97s/it]c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (14,15,16,37,61,62,70) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 29%|████████████████████████                                                            | 2/7 [00:53<02:15, 27.06s/it]c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (15,16,61,62,70) have mixed typ

In [58]:
for f in filenames:
    chunks = pd.read_csv(basedir + f, chunksize=1)
    df = next(chunks)
    print(df.shape)
    display(df)

(1, 77)


,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,ETC_RCD_ITEM_(unknown),새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,SPCMT_(unknown),대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자
0,11305-10019310902,2,집합,3,표제부,서울특별시 강북구 수유동 484-80번지,서울특별시강북구 삼양로77가길 36,삼일교회,11305,10300.0,0.0,484.0,80.0,NaN,NaN,NaN,1,NaN,1.130541e+11,10301.0,0,36.0,0.0,삼일교회,0.0,주건축물,NaN,393.0,233.81,59.49,761.29,761.29,193.71,21.0,철근콘크리트구조,철근콘크리트구조(조적조(4층)),4000,제2종근린생활시설,"근린생활시설, 공동주택(다세대주택)",10.0,(철근)콘크리트,(철근)콘크리트(평스라브),3,0,12.2,4,0,0,0,0,0.0,761.29,0,0.0,0,0.0,0,0.0,7,80.5,20070821,20070829,20101216.0,2007,3080079.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101228


(1, 77)


,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,ETC_RCD_ITEM_(unknown),새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,SPCMT_(unknown),대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자
0,11470-12121,2,집합,3,표제부,서울특별시 양천구 목동 318-41번지,서울특별시 양천구 목동중앙남로16길 48,아주하이빌,11470,10200.0,0.0,318.0,41.0,NaN,NaN,NaN,0,NaN,1.147041e+11,10201.0,0,48.0,0.0,NaN,0.0,주건축물,NaN,261.0,155.82,59.7,605.0,605.0,231.8,21.0,철근콘크리트구조,철근콘크리트조,2000,공동주택,다세대주택,10.0,(철근)콘크리트,평스라브,8,0,14.45,5,0,0,0,0,0.0,605.0,0,0.0,0,0.0,4,51.93,2,23.0,20020409,20020513,20020912.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110415


(1, 77)


...

,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자
0,11290-12818,1,일반,2,일반건축물,서울특별시 성북구 안암동3가 105번지,서울특별시 성북구 인촌로8길 75,NaN,11290,12300.0,0.0,105,0,NaN,NaN,NaN,0,1.129041e+11,12302.0,0,75.0,0.0,NaN,0.0,주건축물,0.0,0.0,0.0,26.45,23.14,0.0,51.0,일반목구조,"일반목구조,기타콘크리트구조",1000,단독주택,주택,20.0,기와,기와,0,1,0.0,1,1,0,0,1,6.62,26.45,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20120222


(1, 75)


,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자
0,11260-21410,1,일반,2,일반건축물,서울특별시 중랑구 중화동 441-2번지,서울특별시 중랑구 봉화산로33길 35,NaN,11260,10300.0,0.0,441,2,NaN,NaN,NaN,0,1.126041e+11,10301.0,0.0,35.0,0.0,NaN,0.0,주건축물,209.0,124.94,59.78,476.31,353.46,169.12,21.0,철근콘크리트구조,철근콘크리트구조,1000,단독주택,단독주택,10.0,(철근)콘크리트,(철근)콘크리트 평슬라브,0,4,10.9,3,1,0,0,0,0.0,0.0,0,0.0,0,0.0,4,122.85,0,0.0,20030117,20030217.0,20030707.0,2003.0,3060079.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20120728


(1, 77)


,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,ETC_RCD_ITEM_(unknown),새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,SPCMT_(unknown),대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자
0,11500-25658,1,일반,2,일반건축물,서울특별시 강서구 화곡동 366-90번지,서울특별시 강서구 가로공원로80가길 34,NaN,11500,10300.0,0.0,366,90,NaN,NaN,NaN,0,1.150041e+11,10301.0,0.0,34.0,0.0,NaN,0.0,주건축물,141.4,69.75,49.32,209.25,139.5,98.67,11.0,벽돌구조,연와조,1000,단독주택,단독주택,10.0,(철근)콘크리트,평스라브 일부 기와,0,1,7.9,2,1,0,0,0,0.0,209.25,0,0.0,0,0.0,0,0.0,0,0.0,19900321,19900521.0,19901116.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20181204,0.0,NaN


In [3]:
basedir = './국토교통부_건축물대장_표제부/'
filenames = [f for f in os.listdir(basedir) if f.endswith('.csv')]
filenames

['bdinfo_2014.csv',
 'bdinfo_2015.csv',
 'bdinfo_2016.csv',
 'bdinfo_2017.csv',
 'bdinfo_2018.csv',
 'bdinfo_2019.csv',
 'bdinfo_2020.csv']

In [4]:
df_2020 = pd.read_csv(basedir + filenames[-1])
print(df_2020.shape)
df_2020.head()

c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (15,58,59,67) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(604844, 77)


,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,ETC_RCD_ITEM_(unknown),새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,SPCMT_(unknown),대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자
0,11500-25658,1,일반,2,일반건축물,서울특별시 강서구 화곡동 366-90번지,서울특별시 강서구 가로공원로80가길 34,NaN,11500,10300.0,0.0,366,90,NaN,NaN,NaN,0,1.150041e+11,10301.0,0.0,34.0,0.0,NaN,0.0,주건축물,141.40,69.75,49.32,209.25,139.50,98.67,11.0,벽돌구조,연와조,01000,단독주택,단독주택,10.0,(철근)콘크리트,평스라브 일부 기와,0,1,7.90,2,1,0,0,0,0.0,209.25,0,0.0,0,0.0,0,0.00,0,0.0,19900321,1.99005e+07,19901116.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20181204,0.0,NaN
1,11200-21285,1,일반,2,일반건축물,서울특별시 성동구 성수동2가 301-27번지,,NaN,11200,11500.0,0.0,301,27,NaN,NaN,NaN,0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,주건축물,90.42,0.00,0.00,206.12,0.00,0.00,21.0,철근콘크리트구조,철근콘크리트 연와조,01000,단독주택,다가구주택(4가구),10.0,(철근)콘크리트,평슬라브위주위기와,0,4,0.00,3,1,0,0,0,0.0,206.12,0,0.0,0,0.0,0,0.00,0,0.0,19900718,NaN,19901220.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20181205,0.0,NaN
2,11170-22196,2,집합,3,표제부,서울특별시 용산구 한강로2가 83-4번지,서울특별시 용산구 한강대로46길 25,NaN,11170,12500.0,0.0,83,4,NaN,NaN,NaN,0,1.117041e+11,12501.0,0.0,25.0,NaN,NaN,0.0,주건축물,219.90,119.72,54.44,536.88,436.51,198.50,21.0,철근콘크리트구조,철근콘크리트구조,04000,제2종근린생활시설,제1.2종근린생활시설,10.0,(철근)콘크리트,평스라브,13,0,13.35,5,1,0,0,0,0.0,536.88,0,0.0,0,0.0,2,39.91,2,23.0,20070215,2.00702e+07,20070731.0,2007.0,3020078.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20191004,1.0,NaN
3,11260-100245871,1,일반,2,일반건축물,서울특별시 중랑구 면목동 455-2번지,서울특별시 중랑구 면목로54길 44,NaN,11260,10100.0,0.0,455,2,NaN,NaN,NaN,0,1.126041e+11,10101.0,0.0,44.0,NaN,NaN,0.0,주건축물,183.10,109.32,59.71,365.32,365.32,199.52,21.0,철근콘크리트구조,철근콘크리트구조,01000,단독주택,단독주택 및 근린생활시설,10.0,(철근)콘크리트,(철근)콘크리트,0,8,13.65,5,0,1,0,0,0.0,365.32,0,0.0,0,0.0,6,69.00,0,0.0,20180614,2.01806e+07,20190104.0,2018.0,3060155.0,건축디자인과,1101.0,신축허가,2,NaN,0.0,0,NaN,0,NaN,0,20191004,0.0,NaN
4,11560-34227,1,일반,2,일반건축물,서울특별시 영등포구 대림동 1057-25번지,서울특별시 영등포구 도림천로11길 15-6,NaN,11560,13300.0,0.0,1057,25,NaN,NaN,NaN,0,1.156042e+11,13301.0,0.0,15.0,6.0,NaN,0.0,주건축물,0.00,53.25,0.00,159.75,106.50,0.00,11.0,벽돌구조,연와조,01000,단독주택,주택,10.0,(철근)콘크리트,스라브,0,1,0.00,2,1,0,0,0,0.0,159.75,0,0.0,0,0.0,0,0.00,0,0.0,19850422,NaN,19850802.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20191004,0.0,NaN


In [7]:
colnames_df = pd.read_excel(basedir + [f for f in os.listdir(basedir) if f.endswith('.xls')][0], header=1)
print(colnames_df.shape)
colnames_df.head()

(77, 3)


,컬럼 한글명,데이터 타입,비고
0,관리_건축물대장_PK,VARCHAR(33),NaN
1,대장_구분_코드,VARCHAR(1),NaN
2,대장_구분_코드_명,VARCHAR(100),NaN
3,대장_종류_코드,VARCHAR(1),NaN
4,대장_종류_코드_명,VARCHAR(100),NaN


In [8]:
colnames_df.tail()

,컬럼 한글명,데이터 타입,비고
72,지능형_건축물_등급,CHAR(1),NaN
73,지능형_건축물_인증점수,NUMERIC(5),NaN
74,생성_일자,VARCHAR(8),NaN
75,내진_설계_적용_여부,VARCHAR(1),NaN
76,내진_능력,VARCHAR(200),NaN


In [9]:
colnames = colnames_df['컬럼 한글명'].tolist()
colnames[-5:]

['지능형_건축물_등급', '지능형_건축물_인증점수', '생성_일자', '내진_설계_적용_여부', '내진_능력']

In [10]:
df_2020_cols = df_2020.columns.tolist()
df_2020_cols.remove('ETC_RCD_ITEM_(unknown)')
df_2020_cols.remove('SPCMT_(unknown)')
df_2020_cols = df_2020_cols + [colnames[-2], colnames[-1]]
df_2020_cols[-5:]

['지능형_건축물_등급', '지능형_건축물_인증점수', '생성_일자', '내진_설계_적용_여부', '내진_능력']

In [11]:
df_2020.columns = df_2020_cols
df_2020.head()

,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,내진_설계_적용_여부,내진_능력
0,11500-25658,1,일반,2,일반건축물,서울특별시 강서구 화곡동 366-90번지,서울특별시 강서구 가로공원로80가길 34,NaN,11500,10300.0,0.0,366,90,NaN,NaN,NaN,0,1.150041e+11,10301.0,0.0,34.0,0.0,NaN,0.0,주건축물,141.40,69.75,49.32,209.25,139.50,98.67,11.0,벽돌구조,연와조,01000,단독주택,단독주택,10.0,(철근)콘크리트,평스라브 일부 기와,0,1,7.90,2,1,0,0,0,0.0,209.25,0,0.0,0,0.0,0,0.00,0,0.0,19900321,1.99005e+07,19901116.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20181204,0.0,NaN
1,11200-21285,1,일반,2,일반건축물,서울특별시 성동구 성수동2가 301-27번지,,NaN,11200,11500.0,0.0,301,27,NaN,NaN,NaN,0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,주건축물,90.42,0.00,0.00,206.12,0.00,0.00,21.0,철근콘크리트구조,철근콘크리트 연와조,01000,단독주택,다가구주택(4가구),10.0,(철근)콘크리트,평슬라브위주위기와,0,4,0.00,3,1,0,0,0,0.0,206.12,0,0.0,0,0.0,0,0.00,0,0.0,19900718,NaN,19901220.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20181205,0.0,NaN
2,11170-22196,2,집합,3,표제부,서울특별시 용산구 한강로2가 83-4번지,서울특별시 용산구 한강대로46길 25,NaN,11170,12500.0,0.0,83,4,NaN,NaN,NaN,0,1.117041e+11,12501.0,0.0,25.0,NaN,NaN,0.0,주건축물,219.90,119.72,54.44,536.88,436.51,198.50,21.0,철근콘크리트구조,철근콘크리트구조,04000,제2종근린생활시설,제1.2종근린생활시설,10.0,(철근)콘크리트,평스라브,13,0,13.35,5,1,0,0,0,0.0,536.88,0,0.0,0,0.0,2,39.91,2,23.0,20070215,2.00702e+07,20070731.0,2007.0,3020078.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20191004,1.0,NaN
3,11260-100245871,1,일반,2,일반건축물,서울특별시 중랑구 면목동 455-2번지,서울특별시 중랑구 면목로54길 44,NaN,11260,10100.0,0.0,455,2,NaN,NaN,NaN,0,1.126041e+11,10101.0,0.0,44.0,NaN,NaN,0.0,주건축물,183.10,109.32,59.71,365.32,365.32,199.52,21.0,철근콘크리트구조,철근콘크리트구조,01000,단독주택,단독주택 및 근린생활시설,10.0,(철근)콘크리트,(철근)콘크리트,0,8,13.65,5,0,1,0,0,0.0,365.32,0,0.0,0,0.0,6,69.00,0,0.0,20180614,2.01806e+07,20190104.0,2018.0,3060155.0,건축디자인과,1101.0,신축허가,2,NaN,0.0,0,NaN,0,NaN,0,20191004,0.0,NaN
4,11560-34227,1,일반,2,일반건축물,서울특별시 영등포구 대림동 1057-25번지,서울특별시 영등포구 도림천로11길 15-6,NaN,11560,13300.0,0.0,1057,25,NaN,NaN,NaN,0,1.156042e+11,13301.0,0.0,15.0,6.0,NaN,0.0,주건축물,0.00,53.25,0.00,159.75,106.50,0.00,11.0,벽돌구조,연와조,01000,단독주택,주택,10.0,(철근)콘크리트,스라브,0,1,0.00,2,1,0,0,0,0.0,159.75,0,0.0,0,0.0,0,0.00,0,0.0,19850422,NaN,19850802.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20191004,0.0,NaN


In [12]:
df_2020.head(2)

,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,내진_설계_적용_여부,내진_능력
0,11500-25658,1,일반,2,일반건축물,서울특별시 강서구 화곡동 366-90번지,서울특별시 강서구 가로공원로80가길 34,NaN,11500,10300.0,0.0,366,90,NaN,NaN,NaN,0,1.150041e+11,10301.0,0.0,34.0,0.0,NaN,0.0,주건축물,141.40,69.75,49.32,209.25,139.5,98.67,11.0,벽돌구조,연와조,01000,단독주택,단독주택,10.0,(철근)콘크리트,평스라브 일부 기와,0,1,7.9,2,1,0,0,0,0.0,209.25,0,0.0,0,0.0,0,0.0,0,0.0,19900321,1.99005e+07,19901116.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20181204,0.0,NaN
1,11200-21285,1,일반,2,일반건축물,서울특별시 성동구 성수동2가 301-27번지,,NaN,11200,11500.0,0.0,301,27,NaN,NaN,NaN,0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,주건축물,90.42,0.00,0.00,206.12,0.0,0.00,21.0,철근콘크리트구조,철근콘크리트 연와조,01000,단독주택,다가구주택(4가구),10.0,(철근)콘크리트,평슬라브위주위기와,0,4,0.0,3,1,0,0,0,0.0,206.12,0,0.0,0,0.0,0,0.0,0,0.0,19900718,NaN,19901220.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20181205,0.0,NaN


In [14]:
df_2020['내진_설계_적용_여부'].describe()

count    331185.000000
mean          0.262741
std           0.440124
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           1.000000
Name: 내진_설계_적용_여부, dtype: float64

In [15]:
df_2020['내진_능력'].describe()

count        13239
unique        2864
top       Ⅶ-0.204g
freq           851
Name: 내진_능력, dtype: object

In [16]:
df_2020['내진_능력'].value_counts()

Ⅶ-0.204g                    851
Ⅶ-0.169g                    518
VII-0.204g                  487
해당없음                        350
Ⅶ-0.170g                    346
                           ... 
vii-0.2253(g)                 1
vll-016g                      1
mmi 7등급0.1699g                1
Ⅷ - 0.528g                    1
MMI등급(Ⅶ) 최대지반가속도(0.169g)      1
Name: 내진_능력, Length: 2864, dtype: int64

In [17]:
filenames

['bdinfo_2014.csv',
 'bdinfo_2015.csv',
 'bdinfo_2016.csv',
 'bdinfo_2017.csv',
 'bdinfo_2018.csv',
 'bdinfo_2019.csv',
 'bdinfo_2020.csv']

In [18]:
df_2020.to_csv(basedir + filenames[-1], index=False)